In [2]:
import pandas as pd

df = pd.read_csv("contracts_classcandidates_0729.csv")
df.head()

,Unnamed: 0,조_항목,항목,원본,조항구분,조항구분_예측,파일명
0,0,목적 및 대상,1조 1항,본 계약은 음식물류 폐기물의 자원화를 효율적으로 추진하기 위하여 필요한 사항을 약정...,['nan'],['필수조항11'],체크1. (과업지시서)2018년 음식물폐기물 민간위탁처리(용역)_계약서.xlsx
1,1,목적 및 대상,1조 2항,위탁처리대상은 [지역명1]군에서 분리 수거한 음식물류폐기물로 한다.,['nan'],['nan'],체크1. (과업지시서)2018년 음식물폐기물 민간위탁처리(용역)_계약서.xlsx
2,2,위탁업무의 내용 및 처리방법,2조 1항,"음식물폐기물의 수집․운반은 “갑”이하고 수집,운반된 음식물류 폐기물의 처리는 “을”...",['nan'],['nan'],체크1. (과업지시서)2018년 음식물폐기물 민간위탁처리(용역)_계약서.xlsx
3,3,위탁업무의 내용 및 처리방법,2조 2항,"“을”은 “갑”이 수집,운반한 음식물류폐기물을 사업장에서 관련 법령에 적법하게 ...",['nan'],['nan'],체크1. (과업지시서)2018년 음식물폐기물 민간위탁처리(용역)_계약서.xlsx
4,4,계약기간,3조 1항,위수탁기간은 [기간]로 한다.,['필수조항1'],['nan'],체크1. (과업지시서)2018년 음식물폐기물 민간위탁처리(용역)_계약서.xlsx


In [3]:
cnt = 0
for i in range(len(df)):
    if df.iloc[i]['조항구분'] == df.iloc[i]['조항구분_예측']:
        cnt += 1
print(f"Accuracy = {cnt / len(df) * 100:.4f}")

Accuracy = 21.8295


## 정답이 후보군에 포함되어 있는 비율 파악

In [167]:
# 1. 각 정답 조항구분 별로 나타나는 조항구분_예측의 구성 형태 파악
# 2. 정답이 후보군에 포함되어 있는지 확인
import re

def evaluate_total(df):
    total = len(df)
    print(f"전체 데이터 수: {total}개")
    cnt = 0
    for i in range(total):
        label, pred = df.iloc[i]['조항구분'], df.iloc[i]['조항구분_예측']
        # 'nan' 또는 '필수조항1' 또는 '필수조항1', '필수조항2'
        labels = [re.sub("'", "", l).strip() for l in label[1:-1].split(',')]
        preds = [re.sub("'", "", p).strip() for p in pred[1:-1].split(',')]
        if set(labels).issubset(set(preds)):
            cnt += 1
    print(f"후보군이 정답을 포함하고 있는 항 수: {cnt}개")
    return cnt / total * 100

    

In [168]:
evaluate_total(df)

전체 데이터 수: 34151개
후보군이 정답을 포함하고 있는 항 수: 16346개


47.86389856812392

In [164]:
# 1. 각 정답 조항구분 별로 나타나는 조항구분_예측의 구성 형태 파악
# 2. 정답이 후보군에 포함되어 있는지 확인
import re

def evaluate(df):
    total = 0
    cnt = 0
    for i in range(len(df)):
        label, pred = df.iloc[i]['조항구분'], df.iloc[i]['조항구분_예측']
        # 'nan' 또는 '필수조항1' 또는 '필수조항1', '필수조항2'
        labels = [re.sub("'", "", l).strip() for l in label[1:-1].split(',')]
        if ['nan'] == labels:
            continue
        if 'nan' in labels:
            labels.remove('nan')
        preds = [re.sub("'", "", p).strip() for p in pred[1:-1].split(',')]
        if set(labels).issubset(set(preds)):
            cnt += 1
        total += 1
    print(f"전체 데이터 수: {total}개")
    print(f"후보군이 정답을 포함하고 있는 항 수: {cnt}개")
    return cnt / total * 100


In [165]:
evaluate(df)

전체 데이터 수: 15325개
후보군이 정답을 포함하고 있는 항 수: 11844개


77.28548123980424

## 정답 조항의 개수에 따른 조항구분 예측 후보의 개수 분포

In [88]:
from collections import defaultdict

def number_of_cands(df):
    cnt_dict = defaultdict(dict)
    total = len(df)
    print(f"전체 데이터 수: {total}개")
    for i in range(total):
        label, pred = df.iloc[i]['조항구분'], df.iloc[i]['조항구분_예측']
        labels = [re.sub("'", "", l).strip() for l in label[1:-1].split(',')]
        preds = [re.sub("'", "", p).strip() for p in pred[1:-1].split(',')]
        # 'nan'인 경우 0개로 취급
        if ['nan'] == labels:
            if ['nan'] == preds:
                if 0 not in cnt_dict[0]:
                    cnt_dict[0][0] = 1
                else:
                    cnt_dict[0][0] += 1
            else:
                if len(preds) not in cnt_dict[0]:
                    cnt_dict[0][len(preds)] = 1
                else:
                    cnt_dict[0][len(preds)] += 1
        elif 'nan' in labels:
            labels.remove('nan')
            if ['nan'] == preds:
                if 0 not in cnt_dict[len(labels)]:
                    cnt_dict[len(labels)][0] = 1
                else:
                    cnt_dict[len(labels)][0] += 1
            else:
                if len(preds) not in cnt_dict[len(labels)]:
                    cnt_dict[len(labels)][len(preds)] = 1
                else:
                    cnt_dict[len(labels)][len(preds)] += 1
        else:
            if len(preds) not in cnt_dict[len(labels)]:
                cnt_dict[len(labels)][len(preds)] = 1
            else:
                cnt_dict[len(labels)][len(preds)] += 1
                
    # sorting
    sorted_cnt_dict = {}
    sorted_by_num_cnt_dict = {}
    
    for k, v_dict in sorted(cnt_dict.items()):
        sorted_cnt_dict[k] = {a:b for a, b in sorted(v_dict.items(), key=lambda x: x[1], reverse=True)}
        sorted_by_num_cnt_dict[k] = {a:b for a, b in sorted(v_dict.items())}

    return sorted_cnt_dict, sorted_by_num_cnt_dict

In [90]:
sorted_cnt_dict, sorted_by_num_cnt_dict = number_of_cands(df)


전체 데이터 수: 34151개


In [92]:
sorted_cnt_dict

{0: {0: 5239,
  1: 4429,
  2: 3058,
  3: 1805,
  4: 1769,
  5: 1080,
  6: 725,
  7: 339,
  8: 199,
  9: 80,
  10: 68,
  11: 19,
  14: 9,
  12: 4,
  13: 3},
 1: {1: 4327,
  4: 1991,
  2: 1844,
  5: 1511,
  6: 1427,
  3: 1319,
  7: 724,
  8: 582,
  9: 330,
  10: 199,
  0: 115,
  11: 62,
  12: 16,
  14: 7,
  13: 6,
  18: 1},
 2: {3: 89,
  5: 82,
  2: 70,
  4: 69,
  1: 51,
  9: 49,
  6: 48,
  7: 48,
  8: 37,
  10: 29,
  12: 19,
  11: 13,
  0: 10,
  13: 5,
  14: 2},
 3: {4: 25,
  5: 24,
  6: 23,
  7: 22,
  2: 20,
  8: 17,
  12: 16,
  3: 16,
  9: 13,
  1: 9,
  10: 8,
  11: 6,
  13: 6,
  15: 1},
 4: {8: 7,
  5: 4,
  4: 4,
  9: 2,
  1: 2,
  11: 2,
  2: 2,
  13: 1,
  12: 1,
  10: 1,
  7: 1,
  6: 1,
  3: 1,
  15: 1},
 5: {9: 2, 7: 1, 13: 1, 11: 1},
 6: {10: 1, 6: 1}}

In [93]:
sorted_by_num_cnt_dict

{0: {0: 5239,
  1: 4429,
  2: 3058,
  3: 1805,
  4: 1769,
  5: 1080,
  6: 725,
  7: 339,
  8: 199,
  9: 80,
  10: 68,
  11: 19,
  12: 4,
  13: 3,
  14: 9},
 1: {0: 115,
  1: 4327,
  2: 1844,
  3: 1319,
  4: 1991,
  5: 1511,
  6: 1427,
  7: 724,
  8: 582,
  9: 330,
  10: 199,
  11: 62,
  12: 16,
  13: 6,
  14: 7,
  18: 1},
 2: {0: 10,
  1: 51,
  2: 70,
  3: 89,
  4: 69,
  5: 82,
  6: 48,
  7: 48,
  8: 37,
  9: 49,
  10: 29,
  11: 13,
  12: 19,
  13: 5,
  14: 2},
 3: {1: 9,
  2: 20,
  3: 16,
  4: 25,
  5: 24,
  6: 23,
  7: 22,
  8: 17,
  9: 13,
  10: 8,
  11: 6,
  12: 16,
  13: 6,
  15: 1},
 4: {1: 2,
  2: 2,
  3: 1,
  4: 4,
  5: 4,
  6: 1,
  7: 1,
  8: 7,
  9: 2,
  10: 1,
  11: 2,
  12: 1,
  13: 1,
  15: 1},
 5: {7: 1, 9: 2, 11: 1, 13: 1},
 6: {6: 1, 10: 1}}

## 정답 조항의 종류에 따른 예측 조항의 분포

In [124]:
from collections import Counter

print(f"전체 데이터 수: {len(df)}")

def predicted_form(df, trg_label):
    counter = Counter()
    cnt = 0
    total = len(df)
    for i in range(total):
        label, pred = df.iloc[i]['조항구분'], df.iloc[i]['조항구분_예측']
        labels = [re.sub("'", "", l).strip() for l in label[1:-1].split(',')]
        preds = [re.sub("'", "", p).strip() for p in pred[1:-1].split(',')]
        if trg_label in labels:
            counter.update(preds)
            cnt += 1
            
    return cnt, counter

전체 데이터 수: 34151


In [125]:
cnt, counter = predicted_form(df, '필수조항1')
print(f"필수조항1이 나오는 횟수는 {cnt}")
print(counter)

필수조항1이 나오는 횟수는 827
Counter({'nan': 545, '필수조항11': 193, '옵션조항3': 193, '필수조항10.1': 48, '옵션조항2': 48, '필수조항10': 47, '필수조항12': 42, '필수조항7': 38, '옵션조항4': 37, '필수조항6': 37, '필수조항8': 32, '필수조항13': 31, '옵션조항7': 27, '옵션조항10': 24, '필수조항4': 24, '옵션조항1': 22, '옵션조항6': 21, '필수조항3': 19, '필수조항2': 18, '필수조항5': 17, '옵션조항5': 15, '옵션조항8': 13, '필수조항9': 9})


In [126]:
cnt, counter = predicted_form(df, '필수조항2')
print(f"필수조항2이 나오는 횟수는 {cnt}")
print(counter)

필수조항2이 나오는 횟수는 147
Counter({'nan': 65, '필수조항2': 55, '필수조항11': 35, '옵션조항3': 29, '옵션조항4': 28, '필수조항9': 16, '필수조항8': 16, '필수조항4': 14, '필수조항10.1': 11, '옵션조항2': 10, '필수조항7': 8, '옵션조항1': 8, '필수조항10': 6, '옵션조항6': 5, '필수조항5': 4, '필수조항3': 3, '필수조항13': 3, '옵션조항7': 2, '필수조항12': 1, '옵션조항5': 1, '옵션조항8': 1, '필수조항6': 1})


### 각 조항별로 예측이 되는 비율 구하기 - 키워드 추가 필요한 조항(?)

In [160]:
label_list = ['필수조항1', '필수조항2', '필수조항3', '필수조항4', '필수조항5', '필수조항6', '필수조항7',
       '필수조항8', '필수조항9', '필수조항10', '필수조항10.1', 
    '필수조항11', '필수조항12', '필수조항13', '옵션조항1', '옵션조항2', '옵션조항3', '옵션조항4', '옵션조항5', 
            '옵션조항6', '옵션조항7', '옵션조항8', '옵션조항9', '옵션조항10', '옵션조항12']

In [161]:
for trg_label in label_list:
    cnt, counter = predicted_form(df, trg_label)
    print(f"{trg_label:>8} --> 정답으로 존재하는 개수: {cnt:4d}개, 예측후보에 존재하는 개수: {counter[trg_label]:4d}개, 비율: {counter[trg_label] / cnt * 100:.4f}%")

   필수조항1 --> 정답으로 존재하는 개수:  827개, 예측후보에 존재하는 개수:    0개, 비율: 0.0000%
   필수조항2 --> 정답으로 존재하는 개수:  147개, 예측후보에 존재하는 개수:   55개, 비율: 37.4150%
   필수조항3 --> 정답으로 존재하는 개수:  738개, 예측후보에 존재하는 개수:  221개, 비율: 29.9458%
   필수조항4 --> 정답으로 존재하는 개수:  839개, 예측후보에 존재하는 개수:  752개, 비율: 89.6305%
   필수조항5 --> 정답으로 존재하는 개수:  761개, 예측후보에 존재하는 개수:  654개, 비율: 85.9396%
   필수조항6 --> 정답으로 존재하는 개수: 1004개, 예측후보에 존재하는 개수:  882개, 비율: 87.8486%
   필수조항7 --> 정답으로 존재하는 개수:  781개, 예측후보에 존재하는 개수:  589개, 비율: 75.4161%
   필수조항8 --> 정답으로 존재하는 개수:  986개, 예측후보에 존재하는 개수:  589개, 비율: 59.7363%
   필수조항9 --> 정답으로 존재하는 개수:  823개, 예측후보에 존재하는 개수:  710개, 비율: 86.2697%
  필수조항10 --> 정답으로 존재하는 개수:  190개, 예측후보에 존재하는 개수:   89개, 비율: 46.8421%
필수조항10.1 --> 정답으로 존재하는 개수:  264개, 예측후보에 존재하는 개수:  145개, 비율: 54.9242%
  필수조항11 --> 정답으로 존재하는 개수: 2538개, 예측후보에 존재하는 개수: 2427개, 비율: 95.6265%
  필수조항12 --> 정답으로 존재하는 개수: 1421개, 예측후보에 존재하는 개수: 1331개, 비율: 93.6664%
  필수조항13 --> 정답으로 존재하는 개수:  258개, 예측후보에 존재하는 개수:  214개, 비율: 82.9457%
   옵션조항1 --> 정답으로 존재하는 개수: 1057개, 예